In [20]:
import requests


In [21]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

In [22]:
data = requests.get(standings_url)


In [23]:
from bs4 import BeautifulSoup


In [24]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]


In [25]:
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])


In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match= "Scores & Fixtures")[0]

In [27]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [28]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()


In [31]:
shooting.columns = shooting.columns.droplevel()

In [32]:
team_data =  matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")

In [ ]:
team_data.head()


In [34]:
years = list (range(2024,2022,-1))
all_matches = []

In [35]:
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"


In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        
        all_matches.append(team_data)
        time.sleep(1)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Only the current season URL
standings_url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"
all_matches = []

# Headers for requests to mimic a browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Get the standings page HTML
data = requests.get(standings_url, headers=headers)
soup = BeautifulSoup(data.text, 'html.parser')

# Find the first standings table
standings_table = soup.select('table.stats_table')[0]

# Extract team URLs
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    print(f"Scraping data for {team_name}...")

    # Fetch the team page
    data = requests.get(team_url, headers=headers)

    # Extract "Scores & Fixtures" table
    try:
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    except ValueError:
        print(f"No 'Scores & Fixtures' table found for {team_name}. Skipping...")
        continue

    # Find the link to the shooting data
    soup = BeautifulSoup(data.text, 'html.parser')
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]

    if not links:
        print(f"No 'Shooting' data found for {team_name}. Skipping...")
        continue

    # Fetch the shooting data
    data = requests.get(f"https://fbref.com{links[0]}", headers=headers)
    try:
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
    except ValueError:
        print(f"No 'Shooting' table found for {team_name}. Skipping...")
        continue

    # Merge match data with shooting data
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        print(f"Data merge failed for {team_name}. Skipping...")
        continue

    # Filter for Premier League matches
    team_data = team_data[team_data["Comp"] == "Premier League"]

    # Add season and team name to the data
    team_data["Season"] = 2024
    team_data["Team"] = team_name

    # Append the data to the all_matches list
    all_matches.append(team_data)

    # Pause to avoid overloading the server
    time.sleep(30)

# After collecting all the data, concatenate the list into a DataFrame
if all_matches:
    final_df = pd.concat(all_matches, ignore_index=True)
    print("Data collection completed successfully.")
else:
    print("No data collected.")


/Users/november/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Scraping data for Manchester City...


/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


Scraping data for Arsenal...


/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


...

Scraping data for Burnley...


/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


Scraping data for Sheffield United...


/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bd/m8wphbrn0dgbjf_jwlvd99700000gn/T/ipykernel_3017/951203121.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


Data collection completed successfully.


In [2]:
len(all_matches)

20

In [3]:
match_df = pd.concat(all_matches)


In [4]:
match_df.columns = [c.lower() for c in match_df.columns]

In [5]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2024,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2024,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2024,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2024,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2024,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2024,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2024,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2024,Sheffield United


In [6]:
match_df.to_csv("matches.csv")